## 다중 쿼리 검색기(MultiQueryRetriever)

- 거리 기반 벡터 데이터베이스 검색은 고차원 공간에서의 쿼리 임베딩(표현)과 '거리'를 기준으로 유사한 임베딩을 가진 문서를 찾는 방식
  
  **하지만 쿼리의 세부적인 차이나 임베딩이 데이터의 의미를 제대로 포착하지 못할 경우, 검색 결과가 달라짐**

  **또한, 이를 수동으로 조정하는 프롬프트 엔지니어링이나 튜닝 작업이 수반**

<br>

#### `MultiQueryRetriever`
-  주어진 사용자 입력 쿼리에 대해 다양한 관점에서 여러 쿼리를 자동으로 생성하는 LLM(Language Learning Model)을 활용해 프롬프트 튜닝 과정을 자동화
-  각각의 쿼리에 대해 관련 문서 집합을 검색하고, 모든 쿼리를 아우르는 고유한 문서들의 합집합을 추출해, 잠재적으로 관련된 더 큰 문서 집합을 얻을 수 있게 함
-  여러 관점에서 동일한 질문을 생성함으로써, `MultiQueryRetriever` 는 거리 기반 검색의 제한을 일정 부분 극복하고, 더욱 풍부한 검색 결과를 제공
  


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

<br>

#### 샘플 벡터 DB

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader = WebBaseLoader(
    "https://teddylee777.github.io/openai/openai-assistant-tutorial/", encoding="utf-8"
)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = loader.load_and_split(text_splitter)

In [4]:
openai_embedding = OpenAIEmbeddings()

db = FAISS.from_documents(docs, openai_embedding)

In [5]:
retriever = db.as_retriever()

query = "OpenAI Assistant API의 Functions 사용법에 대해 알려주세요."
relevant_docs = retriever.invoke(query)

len(relevant_docs)

4

In [6]:
print(relevant_docs[1].page_content)

가장 강력한 도구로서, Assistant에게 사용자 정의 함수를 지정할 수 있습니다. 이는 Chat Completions API에서의 함수 호출과 매우 유사합니다.


Function calling(함수 호출) 도구를 사용하면 Assistant 에게 사용자 정의 함수 를 설명하여 호출해야 하는 함수를 인자와 함께 지능적으로 반환하도록 할 수 있습니다.


Assistant API는 실행 중에 함수를 호출할 때 실행을 일시 중지하며, 함수 호출 결과를 다시 제공하여 Run 실행을 계속할 수 있습니다. (이는 사용자 피드백을 받아 재게할 수 있는 의미이기도 합니다. 아래 튜토리얼에서 상세히 다룹니다).


<br>

### `MultiQueryRetriever`
- 사용할 LLM을 지정하고 질의 생성에 사용하면, retriever가 나머지 작업을 처리

In [7]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

In [8]:
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(),
    llm=llm,
)

In [9]:
question = "OpenAI Assistant API의 Functions 사용법에 대해 알려주세요."
relevant_docs = multiquery_retriever.invoke(question)

In [10]:
print(
    f"===============\n검색된 문서 개수: {len(relevant_docs)}",
    end="\n===============\n",
)

print(relevant_docs[0].page_content)

검색된 문서 개수: 4
OpenAI의 새로운 Assistants API는 대화와 더불어 강력한 도구 접근성을 제공합니다. 본 튜토리얼은 OpenAI Assistants API를 활용하는 내용을 다룹니다. 특히, Assistant API 가 제공하는 도구인 Code Interpreter, Retrieval, Functions 를 활용하는 방법에 대해 다룹니다. 이와 더불어 파일을 업로드 하는 내용과 사용자의 피드백을 제출하는 내용도 튜토리얼 말미에 포함하고 있습니다.



주요내용


<br>

### LCEL Chain 활용
- 질문을 입력 받으면 5개의 질문을 생성한 뒤 "\n" 구분자로 구분하여 생성된 5개 질문을 반환하는 Chain

In [11]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

- 5개의 질문을 생성을 유도하는 프롬프트

In [12]:
prompt = PromptTemplate.from_template(
    """You are an AI language model assistant. 
    Your task is to generate five different versions of the given user question to retrieve relevant documents from a vector database. 
    By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search. 
    Your response should be a list of values separated by new lines, eg: `foo\nbar\nbaz\n`

    #ORIGINAL QUESTION: 
    {question}

    #Answer in Korean:
    """
)

In [13]:
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

In [14]:
custom_multiquery_chain = (
    {"question": RunnablePassthrough()} | prompt | llm | StrOutputParser()
)

In [15]:
question = "OpenAI Assistant API의 Functions 사용법에 대해 알려주세요."

multi_queries = custom_multiquery_chain.invoke(question)
multi_queries

'OpenAI Assistant API의 Functions 기능을 사용하는 방법을 설명해 주세요.  \nOpenAI Assistant API에서 Functions를 활용하는 방법에 대해 알고 싶습니다.  \nOpenAI Assistant API의 Functions를 어떻게 사용할 수 있는지 알려주세요.  \nOpenAI Assistant API의 Functions 사용법에 대한 자세한 정보를 제공해 주세요.  \nOpenAI Assistant API에서 Functions를 사용하는 절차를 설명해 주실 수 있나요?  '

- 이전에 생성한 Chain을 `MultiQueryRetriever` 에 전달하여 `retrieve` 

In [16]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    llm=custom_multiquery_chain, 
    retriever=db.as_retriever()
)

relevant_docs = multiquery_retriever.invoke(question)

print(
    f"===============\n검색된 문서 개수: {len(relevant_docs)}",
    end="\n===============\n",
)

print(relevant_docs[0].page_content)

검색된 문서 개수: 4
OpenAI의 새로운 Assistants API는 대화와 더불어 강력한 도구 접근성을 제공합니다. 본 튜토리얼은 OpenAI Assistants API를 활용하는 내용을 다룹니다. 특히, Assistant API 가 제공하는 도구인 Code Interpreter, Retrieval, Functions 를 활용하는 방법에 대해 다룹니다. 이와 더불어 파일을 업로드 하는 내용과 사용자의 피드백을 제출하는 내용도 튜토리얼 말미에 포함하고 있습니다.



주요내용


<br>

<hr>

<br>

## 다중 벡터저장소 검색기(MultiVectorRetriever)
-  문서를 여러 벡터로 저장하고 관리할 수 있어, 정보 검색의 정확도와 효율성을 대폭 향상
  
<br>

### 문서당 여러 벡터 생성 방법
- **작은 청크 생성**: 문서를 더 작은 단위로 나눈 후, 각 청크에 대해 별도의 임베딩을 생성. 
- **요약 임베딩**: 각 문서의 요약을 생성하고, 이 요약으로부터 임베딩을 생성
  - 문서 전체를 분석하는 대신 핵심적인 요약 부분만을 활용하여 효율성을 극대화
- **가설 질문 활용**: 각 문서에 대해 적합한 가설 질문을 만들고, 이 질문에 기반한 임베딩을 생성
  - 특정 주제나 내용에 대해 깊이 있는 탐색을 원할 때 이 방법이 유용
  - 가설 질문은 문서의 내용을 다양한 관점에서 접근하게 해주며, 더 광범위한 이해를 가능하게 함
- **수동 추가 방식**: 사용자가 문서 검색 시 고려해야 할 특정 질문이나 쿼리를 직접 추가
  - 이 방법을 통해 사용자는 검색 과정에서 보다 세밀한 제어를 할 수 있으며, 자신의 요구 사항에 맞춘 맞춤형 검색이 가능

<br>

- 문서 로드

In [17]:
from langchain_community.document_loaders import PyMuPDFLoader

In [18]:
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

<br>

#### **Chunk + 원본 문서 검색**
- 대용량 정보를 검색하는 경우, 더 작은 단위로 정보를 임베딩하는 것이 유용
- `docstore` 에 원본 문서를 저장하고, `vectorstore` 에 임베딩된 문서를 저장
  
  $\rightarrow$ **문서를 더 작은 단위로 나누어 더 정확한 검색**

In [19]:
import uuid
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers.multi_vector import MultiVectorRetriever

In [20]:
vectorstore = Chroma(
    collection_name="small_bigger_chunks",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small"),
)

# 부모 문서의 저장소
store = InMemoryStore()

In [21]:
id_key = "doc_id"

retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

In [22]:
doc_ids = [str(uuid.uuid4()) for _ in docs]
doc_ids

['518d1350-4b31-4da3-9719-ddff9d9135b6',
 '912f9ed6-7b41-4788-91b4-d91bc6565ba2',
 '17ba052f-02c0-46ed-b053-9af9edf9a287',
 '628cf481-fc1d-4b21-b361-9f30ac4af89e',
 '7a8a3134-2b9d-40fb-8138-8a1c233917bd',
 '6967d97b-d7eb-4e5a-a507-b542c2368cd2',
 '72b99787-a095-4fcd-8838-b95f93c5f9bf',
 '6e2dcaeb-90d6-4c65-8d50-ac4ba5bc76b6',
 'cb8ac9cb-645b-47a7-81a7-0d7555f8427d',
 'ae606a10-469f-4843-8181-efff23b3b7b6',
 'a5e7f9a2-fa9b-4ffd-9a8b-9a0fbb16dbcf',
 '513533fc-627e-4b44-a9b4-c38be97023da',
 'b4e824da-0c34-4ca7-a55d-e1326736c97b',
 '92919e3b-a0ee-41f4-9220-018dfb54001a',
 'cb3cdb5f-e6e0-42af-b7b9-ed97eb0b2891',
 '4e0c1bc9-2db3-461d-9c40-12f1610f08f0',
 '54608c34-449a-464e-a385-351d3fed4138',
 'ca8599c0-be87-49db-b931-22d2bc45e54d',
 'e7e14d75-6141-4243-a98d-0291adb2ae31',
 'e7b2d7b9-42f6-4cfb-bc78-5bcce2aea858',
 '7ad21fd4-9bb4-4077-b5d2-73df5ca7c876',
 'bdf92926-2705-43f6-9839-524bf6e86ff6',
 'aaf99124-430e-458a-88d5-1ebeeb508d29']

- **큰 청크로 분할하기 위한 `parent_text_splitter`, 더 작은 청크로 분할하기 위한 `child_text_splitter`**

In [23]:
parent_text_splitter = RecursiveCharacterTextSplitter(chunk_size=600)
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

- 더 큰 Chunk인 Parent 문서 생성

In [24]:
parent_docs = []

for i, doc in enumerate(docs):
    _id = doc_ids[i]
    
    # 현재 문서를 하위 문서로 분할
    parent_doc = parent_text_splitter.split_documents([doc])

    for _doc in parent_doc:
        # metadata에 문서 ID 를 저장
        _doc.metadata[id_key] = _id
    parent_docs.extend(parent_doc)
    
parent_docs[0].metadata

{'producer': 'Hancom PDF 1.3.0.542',
 'creator': 'Hwp 2018 10.0.0.13462',
 'creationdate': '2023-12-08T13:28:38+09:00',
 'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'total_pages': 23,
 'format': 'PDF 1.4',
 'title': '',
 'author': 'dj',
 'subject': '',
 'keywords': '',
 'moddate': '2023-12-08T13:28:38+09:00',
 'trapped': '',
 'modDate': "D:20231208132838+09'00'",
 'creationDate': "D:20231208132838+09'00'",
 'page': 0,
 'doc_id': '518d1350-4b31-4da3-9719-ddff9d9135b6'}

- 더 작은 Chunk인 Child 문서 생성

In [25]:
child_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    
    # 현재 문서를 하위 문서로 분할
    child_doc = child_text_splitter.split_documents([doc])
    
    for _doc in child_doc:
        # metadata에 문서 ID 를 저장
        _doc.metadata[id_key] = _id
    child_docs.extend(child_doc)

child_docs[0].metadata

{'producer': 'Hancom PDF 1.3.0.542',
 'creator': 'Hwp 2018 10.0.0.13462',
 'creationdate': '2023-12-08T13:28:38+09:00',
 'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'total_pages': 23,
 'format': 'PDF 1.4',
 'title': '',
 'author': 'dj',
 'subject': '',
 'keywords': '',
 'moddate': '2023-12-08T13:28:38+09:00',
 'trapped': '',
 'modDate': "D:20231208132838+09'00'",
 'creationDate': "D:20231208132838+09'00'",
 'page': 0,
 'doc_id': '518d1350-4b31-4da3-9719-ddff9d9135b6'}

In [26]:
print(f"분할된 parent_docs의 개수: {len(parent_docs)}")
print(f"분할된 child_docs의 개수: {len(child_docs)}")

분할된 parent_docs의 개수: 73
분할된 child_docs의 개수: 440


<br>

- 벡터저장소에 새롭게 생성한 작게 쪼개진 하위문서 집합을 추가
- 상위 문서는 생성한 UUID 와 맵핑하여 docstore 에 추가

In [27]:
retriever.vectorstore.add_documents(parent_docs)
retriever.vectorstore.add_documents(child_docs)

retriever.docstore.mset(list(zip(doc_ids, docs)))

- 유사도 검색 수행

In [28]:
relevant_chunks = retriever.vectorstore.similarity_search(
    "삼성전자가 만든 생성형 AI 의 이름은?"
)
print(f"검색된 문서의 개수: {len(relevant_chunks)}")

검색된 문서의 개수: 4


In [29]:
for chunk in relevant_chunks:
    print(chunk.page_content, end="\n\n")
    print(">" * 100, end="\n\n")

☞ 출처 : 삼성전자, ‘삼성 AI 포럼’서 자체 개발 생성형 AI ‘삼성 가우스’ 공개, 2023.11.08.
삼성전자, ‘삼성 개발자 콘퍼런스 코리아 2023’ 개최, 2023.11.14.
TechRepublic, Samsung Gauss: Samsung Research Reveals Generative AI, 2023.11.08.

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··························································· 10
   ▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································ 11

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 

- 원본 문서의 전체 내용을 검색

In [31]:
from langchain.retrievers.multi_vector import SearchType

In [34]:
retriever.search_type = SearchType.mmr

# retriever.search_type = SearchType.similarity
# retriever.search_kwargs = {"k": 1}

In [35]:
relevant_docs = retriever.invoke("삼성전자가 만든 생성형 AI 의 이름은?")
print(f"검색된 문서의 개수: {len(relevant_docs)}", end="\n\n")
print("=" * 100, end="\n\n")
print(relevant_docs[0].page_content)

검색된 문서의 개수: 3


SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 바꿀 수 있도록 지원하며 
저해상도 이미지의 고해상도 전환도 지원
n IT 전문지 테크리퍼블릭(TechRepublic)은 온디바이스 AI가 주요 기술 트렌드로 부상했다며, 
2024년부터 가우스를 탑재한 삼성

<br>

### 요약본(summary)을 벡터저장소에 저장
- 요약은 종종 청크(chunk)의 내용을 보다 정확하게 추출할 수 있어 더 나은 검색 결과를 얻을 수 있음

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [37]:
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=50)
split_docs = loader.load_and_split(text_splitter)

print(f"분할된 문서의 개수: {len(split_docs)}")

분할된 문서의 개수: 61


In [38]:
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

- `docs` 리스트의 문서들을 일괄 요약

In [39]:
summary_chain = (
    {"doc": lambda x: x.page_content}

    | ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert in summarizing documents in Korean."),
            (
                "user",
                "Summarize the following documents in 3 sentences in bullet points format.\n\n{doc}",
            ),
        ]
    )

    | ChatOpenAI(temperature=0, model="gpt-4o-mini")
    | StrOutputParser()
)

In [40]:
summaries = summary_chain.batch(split_docs, {"max_concurrency": 10})

In [41]:
len(summaries)

61

In [ ]:
print(split_docs[33].page_content, end="\n\n")

print("[요약]")
print(summaries[33])

SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에

[요약]
- 삼성전자가 온디바이스에서 작동 가능한 생성 AI 모델 '삼성 가우스'를 공개하였으며, 이 모델은 언어, 코드, 이미지의 3개 모델로 구성되어 있다.
- '삼성 가우스'는 정규분포 이론을 정립한 수학자 가우스의 이름을 따왔으며, 다양한 상황에 최적화된 모델 선택이 가능하다.
- 삼성전자는 이 AI 모델이 사용자 정보를 외부로 유출하지 않도록 설계되었으며, 향후 다양한 제품에 단계적으로 탑재할 계획이다.


<br>

- 자식 청크(child chunks)를 인덱싱

In [43]:
import uuid

In [44]:
summary_vectorstore = Chroma(
    collection_name="summaries",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small"),
)

In [45]:
store = InMemoryStore()
id_key = "doc_id"

retriever = MultiVectorRetriever(
    vectorstore=summary_vectorstore,  # 벡터 저장소
    byte_store=store,  # 바이트 저장소
    id_key=id_key,  # 문서 ID 키
)

doc_ids = [str(uuid.uuid4()) for _ in split_docs]

- 요약된 문서와 메타데이터 저장
  - 요약된 내용을 페이지 콘텐츠로 하고, 문서 ID를 메타데이터로 포함하는 Document 객체

In [47]:
summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

In [49]:
len(summary_docs)

61

- 요약된 문서를 벡터 저장소에 추가


In [ ]:
retriever.vectorstore.add_documents(
    summary_docs
)

retriever.docstore.mset(list(zip(doc_ids, split_docs)))

- 유사도 검색

In [53]:
result_docs = summary_vectorstore.similarity_search(
    "삼성전자가 만든 생성형 AI 의 이름은?"
)

In [54]:
print(result_docs[0].page_content)

- 삼성전자가 온디바이스에서 작동 가능한 생성 AI 모델 '삼성 가우스'를 공개하였으며, 이 모델은 언어, 코드, 이미지의 3개 모델로 구성되어 있다.
- '삼성 가우스'는 정규분포 이론을 정립한 수학자 가우스의 이름을 따왔으며, 다양한 상황에 최적화된 모델 선택이 가능하다.
- 삼성전자는 이 AI 모델이 사용자 정보를 외부로 유출하지 않도록 설계되었으며, 향후 다양한 제품에 단계적으로 탑재할 계획이다.


- 질문과 관련된 문서를 검색

In [55]:
retrieved_docs = retriever.invoke("삼성전자가 만든 생성형 AI 의 이름은?")
print(retrieved_docs[0].page_content)

SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에


<br>

### 가설 쿼리(Hypothetical Queries) 를 활용하여 문서 내용 탐색
- LLM은 특정 문서에 대해 가정할 수 있는 질문 목록을 생성하는 데에도 사용
- 질문들은 임베딩(embedding)될 수 있으며, 이를 통해 문서의 내용을 더욱 깊이 있게 탐색하고 이해
- 가정 질문 생성은 문서의 주요 주제와 개념을 파악하는 데 도움이 되며, 독자들이 문서 내용에 대해 더 많은 궁금증을 갖도록 유도

In [58]:
functions = [
    {
        "name": "hypothetical_questions",  # 함수의 이름을 지정
        "description": "Generate hypothetical questions",  # 함수에 대한 설명
        "parameters": {  # 함수의 매개변수를 정의
            "type": "object",  # 매개변수의 타입을 객체로 지정
            "properties": {  # 객체의 속성을 정의
                "questions": {  # 'questions' 속성을 정의
                    "type": "array",  # 'questions'의 타입을 배열로 지정
                    "items": {
                        "type": "string"
                    },  # 배열의 요소 타입을 문자열로 지정
                },
            },
            "required": ["questions"],  # 필수 매개변수로 'questions'를 지정
        },
    }
]

- 주어진 문서를 기반으로 3개의 가상 질문을 생성하는 프롬프트 템플릿 & 체인 생성

In [57]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain_openai import ChatOpenAI

In [59]:
hypothetical_query_chain = (
    {"doc": lambda x: x.page_content}
    
    # 아래 문서를 사용하여 답변할 수 있는 가상의 질문을 정확히 3개 생성하도록 요청
    | ChatPromptTemplate.from_template(
        "Generate a list of exactly 3 hypothetical questions that the below document could be used to answer. "
        "Potential users are those interested in the AI industry. Create questions that they would be interested in. "
        "Output should be written in Korean:\n\n{doc}"
    )
    
    | ChatOpenAI(max_retries=0, model="gpt-4o-mini").bind(
        functions=functions, function_call={"name": "hypothetical_questions"}
    )
    
    # 출력에서 "questions" 키에 해당하는 값을 추출
    | JsonKeyOutputFunctionsParser(key_name="questions")
)

- 문서 목록에 대해 가설 질문 생성

In [63]:
hypothetical_questions = hypothetical_query_chain.batch(
    split_docs, {"max_concurrency": 10}
)

hypothetical_questions[33]

['삼성 가우스가 다른 AI 모델과 비교해 경쟁력이 있는 이유는 무엇일까요?',
 '온디바이스에서 작동 가능하다는 점이 삼성 가우스에 어떤 이점을 제공할까요?',
 '삼성전자가 삼성 가우스를 통해 AI 산업에 미칠 영향은 어떤 것들이 있을까요?']

<br>

- 생성한 가설 쿼리(Hypothetical Queries) 를 벡터저장소에 저장

In [64]:
hypothetical_vectorstore = Chroma(
    collection_name="hypo-questions", embedding_function=OpenAIEmbeddings()
)

store = InMemoryStore()

id_key = "doc_id"

retriever = MultiVectorRetriever(
    vectorstore=hypothetical_vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in split_docs]

In [65]:
question_docs = []

for i, question_list in enumerate(hypothetical_questions):
    question_docs.extend(
        [Document(page_content=s, metadata={id_key: doc_ids[i]}) for s in question_list]
    )

In [67]:
question_docs[:5]

[Document(metadata={'doc_id': 'e611ab51-3756-4f51-8e07-b79bfa9e208a'}, page_content='AI 산업의 향후 발전 방향은 어떻게 될까요?'),
 Document(metadata={'doc_id': 'e611ab51-3756-4f51-8e07-b79bfa9e208a'}, page_content='AI 기술이 다양한 산업에 미치는 영향은 무엇인가요?'),
 Document(metadata={'doc_id': 'e611ab51-3756-4f51-8e07-b79bfa9e208a'}, page_content='2023년에 AI 관련 기업들이 주목해야 할 트렌드는 무엇일까요?'),
 Document(metadata={'doc_id': 'e5d8630f-55fd-46dd-8241-e8556046150f'}, page_content='현재 AI 산업에서 정책 및 법제 변화가 기업의 운영에 어떤 영향을 미칠 수 있는가?'),
 Document(metadata={'doc_id': 'e5d8630f-55fd-46dd-8241-e8556046150f'}, page_content='AI 안전성에 대한 국제적 합의가 기업의 혁신과 발전에 어떤 긍정적인 영향을 미칠 수 있을까?')]

<br>

- 가설 쿼리를 문서에 추가

In [ ]:
retriever.vectorstore.add_documents(question_docs)
retriever.docstore.mset(list(zip(doc_ids, split_docs)))

- 유사도 검색

In [69]:
result_docs = hypothetical_vectorstore.similarity_search(
    "삼성전자가 만든 생성형 AI 의 이름은?"
)

In [70]:
for doc in result_docs:
    print(doc.page_content)
    print(doc.metadata)

삼성전자가 발표한 Generative AI 기술은 경쟁사와 어떤 차별점을 가지고 있나요?
{'doc_id': '10d78f8f-340f-433e-9486-1ca111ed7986'}
삼성전자의 생성 AI '삼성 가우스'가 시장의 경쟁 구도에 어떻게 변화시킬 가능성이 있을까?
{'doc_id': 'bde06513-4c52-449c-a2d1-f7df2c5ba84d'}
삼성전자가 삼성 가우스를 통해 AI 산업에 미칠 영향은 어떤 것들이 있을까요?
{'doc_id': '4f36138d-80fd-472e-848e-b17d8ea29f68'}
삼성 개발자 콘퍼런스 코리아 2023에서 발표된 내용이 AI 산업에 어떠한 영향을 미칠까요?
{'doc_id': '10d78f8f-340f-433e-9486-1ca111ed7986'}


In [ ]:
retrieved_docs = retriever.invoke(result_docs[1].page_content)

for doc in retrieved_docs:
    print(doc.page_content)

▹ 코히어, 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개  ······································· 8
   ▹ 알리바바 클라우드, 최신 LLM ‘통이치엔원 2.0’ 공개 ······················································ 9
   ▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··························································· 10
   ▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································ 11
   ▹ IDC, 2027년 AI 소프트웨어 매출 2,500억 달러 돌파 전망··········································· 12
   ▹ 빌 게이츠, AI 에이전트로 인한 컴퓨터 사용의 패러다임 변화 전망································ 13
SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지

<br>

<hr>

## 셀프 쿼리 검색기 (`SelfQueryRetriever`)
- **자체적으로 질문을 생성하고 해결할 수 있는 기능을 갖춘 검색 도구**
- **사용자가 제공한 자연어 질의를 바탕으로, query-constructing LLM chain을 사용해 구조화된 질의를 생성**
  
  $\rightarrow$ **이 구조화된 질의를 기본 벡터 데이터 저장소(VectorStore)에 적용하여 검색을 수행**

- **단순히 사용자의 입력 질의를 저장된 문서의 내용과 의미적으로 비교하는 것을 넘어서, 사용자의 질의에서 문서의 메타데이터에 대한 필터를 추출 하고, 이 필터를 실행하여 관련된 문서를 탐색**
  


<br>

- 샘플 데이터 생성

In [19]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

In [20]:
docs = [
    Document(
        page_content="수분 가득한 히알루론산 세럼으로 피부 속 깊은 곳까지 수분을 공급합니다.",
        metadata={"year": 2024, "category": "스킨케어", "user_rating": 4.7},
    ),
    Document(
        page_content="24시간 지속되는 매트한 피니시의 파운데이션, 모공을 커버하고 자연스러운 피부 표현이 가능합니다.",
        metadata={"year": 2023, "category": "메이크업", "user_rating": 4.5},
    ),
    Document(
        page_content="식물성 성분으로 만든 저자극 클렌징 오일, 메이크업과 노폐물을 부드럽게 제거합니다.",
        metadata={"year": 2023, "category": "클렌징", "user_rating": 4.8},
    ),
    Document(
        page_content="비타민 C 함유 브라이트닝 크림, 칙칙한 피부톤을 환하게 밝혀줍니다.",
        metadata={"year": 2023, "category": "스킨케어", "user_rating": 4.6},
    ),
    Document(
        page_content="롱래스팅 립스틱, 선명한 발색과 촉촉한 사용감으로 하루종일 편안하게 사용 가능합니다.",
        metadata={"year": 2024, "category": "메이크업", "user_rating": 4.4},
    ),
    Document(
        page_content="자외선 차단 기능이 있는 톤업 선크림, SPF50+/PA++++ 높은 자외선 차단 지수로 피부를 보호합니다.",
        metadata={"year": 2024, "category": "선케어", "user_rating": 4.9},
    ),
]

vectorstore = Chroma.from_documents(
    docs, OpenAIEmbeddings(model="text-embedding-3-small")
)

<br>

<hr>